<a href="https://colab.research.google.com/github/varanasiakhila/AI-interviewer/blob/main/AI_Interviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip uninstall -y fitz
!pip install pymupdf gradio deepface opencv-python-headless

In [2]:
!pip install pymupdf

In [ ]:
!pip install gradio pdfplumber python-docx spacy opencv-python-headless fer
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [5]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 19.2 MB/s eta 0:00:00
  Attempting uninstall: gradio
    Found existing installation: gradio 5.38.2
    Uninstalling gradio-5.38.2:
      Successfully uninstalled gradio-5.38.2


In [6]:
pip install gradio pdfplumber SpeechRecognition fer librosa numpy gTTS pydub Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [7]:
!pip install docx2txt

In [9]:
import gradio as gr
import pdfplumber, re, random, tempfile, os
import spacy
import numpy as np
from gtts import gTTS
import docx2txt

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Global session data
session = {
    "keywords": [], "questions": [], "qcount": 0,
    "video_path": None, "emotions": []
}

# ------- Resume Parsing -------
def extract_keywords(text):
    sections = re.split(r'\n(?=[A-Z][A-Za-z ]+)', text)
    entries = []
    for sec in sections:
        hdr = sec.split("\n", 1)[0]
        if re.search(r"(skills|technical skills|tools|technologies)", hdr, re.I):
            entries += re.findall(r"\b[A-Za-z0-9+#\.-]+\b", sec)
    doc = nlp(" ".join(entries))
    return [tok.text for tok in doc if tok.pos_ in ["NOUN", "PROPN"]]

def parse_resume(file):
    if file is None:
        return "⚠️ Please upload a valid resume."

    ext = os.path.splitext(file.name)[-1].lower()
    temp_path = tempfile.NamedTemporaryFile(delete=False, suffix=ext)
    temp_path.write(file.read())
    temp_path.close()

    try:
        if ext == ".pdf":
            with pdfplumber.open(temp_path.name) as pdf:
                text = "\n".join([p.extract_text() or "" for p in pdf.pages])
        elif ext == ".docx":
            text = docx2txt.process(temp_path.name)
        else:
            return "❌ Unsupported file format. Upload PDF or DOCX."

        kws = extract_keywords(text)
        session.update({
            "keywords": kws,
            "questions": [],
            "qcount": 0,
            "video_path": None,
            "emotions": []
        })

        # Question generation
        if kws:
            random.shuffle(kws)
            templates = [
                "Can you explain your experience with {}?",
                "How have you used {} in your past projects?",
                "What challenges did you face when working with {}?",
                "How confident are you with {}?",
                "Have you applied {} in a real-world scenario?"
            ]
            tech_qs = [random.choice(templates).format(kw) for kw in kws[:5]]
        else:
            tech_qs = ["Tell me about your technical strengths."]

        hr_qs = random.sample([
            "Tell me about yourself.",
            "What are your strengths and weaknesses?",
            "Why do you want to work with us?",
            "Describe a challenge you faced and how you overcame it.",
            "Where do you see yourself in five years?",
            "What motivates you at work?",
            "How do you handle pressure and deadlines?",
            "How do you handle feedback or criticism?"
        ], 5)

        session["questions"] = tech_qs + hr_qs
        return "✅ Extracted keywords:\n" + ", ".join(kws)
    finally:
        os.remove(temp_path.name)

# ------- TTS Question Playback -------
def speak_question():
    idx = session["qcount"]
    if idx >= len(session["questions"]):
        return "🎉 Interview complete. Please click 'Stop Interview'.", None
    q = session["questions"][idx]
    tts = gTTS(text=q)
    tts_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(tts_path.name)
    return q, tts_path.name

# ------- Next Question -------
def next_question():
    if session["qcount"] < len(session["questions"]) - 1:
        session["qcount"] += 1
        return speak_question()
    return "🎉 All questions asked. Click Stop Interview.", None

# ------- Stop Interview -------
def stop_interview(video):
    if video is None:
        return "⚠️ No video uploaded.", None
    session["video_path"] = video.name
    return "🛑 Interview stopped — processing video...", None

# ------- Simulated Emotion Analysis -------
def analyze_emotion_from_video(video_path):
    total_frames = random.randint(100, 300)
    emotions = ["happy", "neutral", "angry", "fear", "surprise", "sad", "disgust"]
    allocated = [random.randint(0, total_frames // 3) for _ in emotions]
    scale = total_frames / sum(allocated)
    emotion_counts = {e: int(c * scale) for e, c in zip(emotions, allocated)}

    # Adjust difference
    diff = total_frames - sum(emotion_counts.values())
    if diff != 0:
        emotion_counts["neutral"] += diff

    summary = "\n".join([
        f"🙂 Happy: {emotion_counts['happy']} frames",
        f"😐 Neutral: {emotion_counts['neutral']} frames",
        f"😠 Angry: {emotion_counts['angry']} frames",
        f"😟 Fear: {emotion_counts['fear']} frames",
        f"🎭 Others: {total_frames - sum(emotion_counts[e] for e in ['happy', 'neutral', 'angry', 'fear'])} frames",
        f"📊 Total frames: {total_frames}"
    ])

    top_emotion = max(emotion_counts, key=emotion_counts.get)
    emotion_variation = ', '.join(f"{e} ({c})" for e, c in sorted(emotion_counts.items(), key=lambda x: -x[1]))

    tip = random.choice([
        "Smile more for warmth.",
        "Relax before answering.",
        "Keep good eye contact.",
        "Pause briefly before responding.",
        "Practice in front of a mirror."
    ])

    feedback = f"""
🧠 Top emotion: {top_emotion}
✅ Smiles: {emotion_counts['happy']} frames
⚠ Tension: {emotion_counts['fear']} | Anger: {emotion_counts['angry']}
🌈 Emotion spread: {emotion_variation}
💡 Tip: {tip}
"""
    return summary, feedback.strip()

def analyze_and_feedback():
    if not session["video_path"]:
        return "⚠️ No video uploaded for analysis.", ""
    summary, feedback = analyze_emotion_from_video(session["video_path"])
    return summary, feedback

# ------- Gradio UI -------
with gr.Blocks() as app:
    gr.Markdown("# 🤖 Interview AI — Technical + Emotion Feedback")

    with gr.Tab("📄 Resume Parsing"):
        pdf_upload = gr.File(file_types=[".pdf", ".docx"], label="Upload Resume")
        skills_out = gr.Textbox(label="Extracted Skills & Keywords")
        parse_btn = gr.Button("Parse Resume")
        parse_btn.click(parse_resume, inputs=pdf_upload, outputs=skills_out)

    with gr.Tab("🎥 Interview"):
        with gr.Row():
            video_in = gr.Video(label="Upload Interview Video (.mp4)")
            with gr.Column():
                q_text = gr.Textbox(label="Current Question")
                q_audio = gr.Audio(label="Question (TTS Audio)", type="filepath")
                start_btn = gr.Button("▶ Start")
                next_btn = gr.Button("➡ Next")
                stop_btn = gr.Button("🛑 Stop")
                feedback_btn = gr.Button("📊 Analyze")

        status = gr.Textbox(label="Status")
        feedback = gr.Textbox(label="Emotion Feedback", lines=8)

        start_btn.click(speak_question, outputs=[q_text, q_audio])
        next_btn.click(next_question, outputs=[q_text, q_audio])
        stop_btn.click(stop_interview, inputs=video_in, outputs=[status, q_audio])
        feedback_btn.click(analyze_and_feedback, outputs=[status, feedback])

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://529117408adb66adb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
